## NN

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_pickle("data_with_embeddings.pkl")
test_df = pd.read_pickle("test_data_with_embeddings.pkl")

In [3]:
X_train = list(train_df["embedding"])
y_train = train_df["sentiment_class"]
X_test = list(test_df["embedding"])
y_test = test_df["sentiment_class"]

print(len(X_train))
print(len(X_train[0]))
print(len(X_test))

27480
768
3534


In [4]:
import numpy as np
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_train)
y_test = np.array(y_train)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [6]:
device = "cuda"

In [7]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

In [8]:
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)

In [9]:
class NN(nn.Module):
    def __init__(self, input_dim):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)  
        self.fc2 = nn.Linear(512, 256)         
        self.fc3 = nn.Linear(256, 128)         
        self.fc4 = nn.Linear(128, 64)          
        self.fc5 = nn.Linear(64, 3)   
        self.gelu = nn.GELU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.gelu(self.fc1(x))
        x = self.gelu(self.fc2(x))
        x = self.gelu(self.fc3(x))
        x = self.gelu(self.fc4(x))
        x = self.fc5(x)
        return x
print(len(X_train[0]))

768


In [10]:
model = NN(input_dim=len(X_train[0])).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        X_batch, y_batch = batch
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = loss_function(outputs.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.3641216456890106
Epoch 2, Loss: 0.4677952527999878
Epoch 3, Loss: 0.5310391187667847
Epoch 4, Loss: 0.4388793706893921
Epoch 5, Loss: 0.41193705797195435
Epoch 6, Loss: 0.32082995772361755
Epoch 7, Loss: 0.30325859785079956
Epoch 8, Loss: 0.30541402101516724
Epoch 9, Loss: 0.2734943628311157
Epoch 10, Loss: 0.24180345237255096
Epoch 11, Loss: 0.37968939542770386
Epoch 12, Loss: 0.4422644376754761
Epoch 13, Loss: 0.20456074178218842
Epoch 14, Loss: 0.21106351912021637
Epoch 15, Loss: 0.05848437920212746
Epoch 16, Loss: 0.06673743575811386
Epoch 17, Loss: 0.14521242678165436
Epoch 18, Loss: 0.08019820600748062
Epoch 19, Loss: 0.0543985478579998
Epoch 20, Loss: 0.0698603019118309
Epoch 21, Loss: 0.03990492969751358
Epoch 22, Loss: 0.021909642964601517
Epoch 23, Loss: 0.036552026867866516
Epoch 24, Loss: 0.07765203714370728
Epoch 25, Loss: 0.04688672721385956
Epoch 26, Loss: 0.07496756315231323
Epoch 27, Loss: 0.0705350935459137
Epoch 28, Loss: 0.03933535888791084
Epoch 29

In [13]:
model.eval()

NN(
  (fc1): Linear(in_features=768, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=3, bias=True)
  (gelu): GELU(approximate='none')
  (sigmoid): Sigmoid()
)

In [15]:
import torch
from sklearn.metrics import accuracy_score

# Ensure labels are in the correct format for multi-class classification
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)  # Use torch.long for multi-class labels

batch_size = 64

y_true = []
y_pred = []

num_samples = len(X_test_tensor)  # Number of samples, not features
for i in range(0, num_samples, batch_size):
    # Get the batch
    X_batch = X_test_tensor[i:i+batch_size].to(device)
    y_batch = y_test_tensor[i:i+batch_size].to(device)
    
    # Forward pass to get predictions
    with torch.no_grad():
        outputs = model(X_batch)
        
    # Convert outputs to predicted class indices (class with the highest probability)
    predicted = torch.argmax(outputs, dim=1)  # Get the class with the highest probability
    
    # Append true labels and predicted labels
    y_true.extend(y_batch.cpu().numpy())  # Move to CPU if necessary
    y_pred.extend(predicted.cpu().numpy())  # Move to CPU if necessary

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)

# Print accuracy
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 98.54%
